# Books Recommender System - Collaborative Based Clustering

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Importing Data

In [4]:
books_df = pd.read_csv('BX-Books.csv', sep=';',error_bad_lines=False,encoding='latin-1')
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';',error_bad_lines=False,encoding='latin-1')
users = pd.read_csv('BX-Users.csv', sep=';',error_bad_lines=False,encoding='latin-1' )

C:\Users\PC\AppData\Local\Temp\ipykernel_1408\984414932.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books_df = pd.read_csv('BX-Books.csv', sep=';',error_bad_lines=False,encoding='latin-1')
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: exp

### Data Preprocessing

In [5]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
books = books_df[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]


In [7]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [8]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [9]:
# Rename the colunms names of the three dataframes
users.rename(columns = {'User-ID':'id',
                        'Location':'location',
                        'Age':'age'}, inplace = True)

ratings.rename(columns = {'User-ID':'id',
                          'Book-Rating':'rating'}, inplace = True)

books.rename(columns = {'Book-Title':'title', 
                        'Book-Author':'author',
                        'Year-Of-Publication':'year',
                        'Publisher':'publisher',
                       'Image-URL-L':'image_url'}, inplace = True)

C:\Users\PC\AppData\Local\Temp\ipykernel_1408\762728356.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.rename(columns = {'Book-Title':'title',


In [10]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ISBN       271360 non-null  object
 1   title      271360 non-null  object
 2   author     271359 non-null  object
 3   year       271360 non-null  object
 4   publisher  271358 non-null  object
 5   image_url  271357 non-null  object
dtypes: object(6)
memory usage: 12.4+ MB


In [11]:
ratings['id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: id, Length: 105283, dtype: int64

In [12]:
# We will only keep users who had voted for 100 books or more
users_100 = ratings['id'].value_counts() >= 100
users_ids = users_100[users_100].index
ratings = ratings[ratings['id'].isin(users_ids)]

In [13]:
ratings.head()

,id,ISBN,rating
412,276925,0006511929,0
413,276925,002542730X,10
414,276925,0060520507,0
415,276925,0060930934,0
416,276925,0060951303,0


In [14]:
ratings.shape

(658805, 3)

In [15]:
no_ratings = ratings.ISBN.value_counts().reset_index()
no_ratings.rename(columns = {'index':'ISBN',
                             'ISBN':'no_ratings'}, inplace = True)
no_ratings.head()

,ISBN,no_ratings
0,0971880107,617
1,0316666343,430
2,0385504209,325
3,0060928336,306
4,044021145X,276


In [16]:
# Filtering fbooks that have less than 30 ratings
no_ratings = no_ratings.query(" no_ratings >= 30 ")
no_ratings = no_ratings.merge(ratings, on = 'ISBN')
books = books.merge(no_ratings, on='ISBN')
books = books[['id','ISBN','title','author','no_ratings','rating','year','publisher','image_url']]
books.head()

,id,ISBN,title,author,no_ratings,rating,year,publisher,image_url
0,277478,0440234743,The Testament,John Grisham,177,0,1999,Dell,http://images.amazon.com/images/P/0440234743.0...
1,2977,0440234743,The Testament,John Grisham,177,0,1999,Dell,http://images.amazon.com/images/P/0440234743.0...
2,3363,0440234743,The Testament,John Grisham,177,0,1999,Dell,http://images.amazon.com/images/P/0440234743.0...
3,7346,0440234743,The Testament,John Grisham,177,9,1999,Dell,http://images.amazon.com/images/P/0440234743.0...
4,9747,0440234743,The Testament,John Grisham,177,0,1999,Dell,http://images.amazon.com/images/P/0440234743.0...


In [17]:
# Dropping Duplicates
books = books.drop_duplicates(['ISBN','id'])

In [20]:
book_pivot = books.pivot_table(columns='id', index='title', values= 'rating')
book_pivot

id,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,276018,276463,276680,276925,277427,277478,277639,278137,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16 Lighthouse Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010: Odyssey Two,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
You Belong to Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
You Belong to Me and Other True Cases (Ann Rule's Crime Files: Vol. 2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
book_pivot.fillna(0, inplace=True)

## Train the model

In [85]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

In [26]:
book_sparse = csr_matrix(book_pivot)
model = NearestNeighbors(algorithm= 'brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [48]:
book_pivot.iloc[21,:]

id
254       0.0
507       0.0
882       0.0
1424      0.0
1435      0.0
         ... 
277478    0.0
277639    0.0
278137    0.0
278188    0.0
278418    0.0
Name: A Cry In The Night, Length: 1799, dtype: float64

In [51]:
distances, suggestions = model.kneighbors(book_pivot.iloc[21,:].values.reshape(1,-1), n_neighbors=6 )
print("Distance ",distances)
print("Suggestion ",suggestions)

Distance  [[ 0.         28.4956137  28.60069929 28.68797658 28.72281323 29.12043956]]
Suggestion  [[ 21 649 954  60  99 886]]


In [84]:
# Get the suggested books
suggested_books = []
book_names = book_pivot.index
for i in suggestions:
    suggested_books.append(book_names[i].tolist())
suggested_books

[['A Cry In The Night',
  'If Tomorrow Comes',
  "Passion's Promise",
  'A Theory of Relativity',
  'All I Need Is You',
  "Night Moves (Tom Clancy's Net Force, No. 3)"]]

In [92]:
urls = []
for names in suggested_books:
    for name in names:
        urls.append(books.query("title == @name ")['image_url'].unique()[0])
urls

['http://images.amazon.com/images/P/0671886665.01.LZZZZZZZ.jpg',
 'http://images.amazon.com/images/P/0446329894.01.LZZZZZZZ.jpg',
 'http://images.amazon.com/images/P/0440129265.01.LZZZZZZZ.jpg',
 'http://images.amazon.com/images/P/0061031992.01.LZZZZZZZ.jpg',
 'http://images.amazon.com/images/P/0380762609.01.LZZZZZZZ.jpg',
 'http://images.amazon.com/images/P/042517400X.01.LZZZZZZZ.jpg']

In [107]:
def recommend_book(title):
    suggested_books = []
    book_id = np.where(book_pivot.index == title)[0][0]
    distances, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in suggestions:
        suggested_books.append(book_names[i].tolist())
        for j in np.nditer(suggested_books):
            if j== title:
                print(f"You searched '{title}'\n")
                print("The suggestion books are: \n")
            else:
                print(j)

In [109]:
book_name = "If Tomorrow Comes"
recommend_book(book_name)

You searched 'If Tomorrow Comes'

The suggestion books are: 

A Theory of Relativity
Night Moves (Tom Clancy's Net Force, No. 3)
The Least Likely Bride
Stay Out of the Basement (Goosebumps, No 2)
Hurricane Bay


In [110]:
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(books,open('artifacts/books.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))